In [1]:
import torch.nn.functional as F
import torch.optim as optim
import torch
import torch.nn as nn
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

ModuleNotFoundError: No module named 'torchvision'

In [ ]:
regressiondatafianl = pd.read_csv('data/regressiontrainning_data.csv',index_col = 0)
regressiondatafianl.head()

In [ ]:
features,targets_2 = torch.from_numpy(regressiondatafinal.drop('revenue_30', axis=1).values).float(),torch.from_numpy(regressiondatafinal['revenue_30'].values.reshape(-1,1)).float()

data_r = [(x,y) for x,y in zip(features, targets_2)]

valid_size =.2

num_train = len(data_r)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

batch_size = 32
num_workers = 0
train_loader = torch.utils.data.DataLoader(data_r, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(data_r, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)

In [ ]:

# define the CNN architecture
class Net(nn.Module):
    def __init__(self,in_feature,input_nodes, hidden_nodes, output_nodes):
        super(Net, self).__init__() #

        self.fc1 = nn.Linear(in_feature,input_nodes)
        self.fc2 = nn.Linear(input_nodes,hidden_nodes)
        self.fc3 = nn.Linear(hidden_nodes,output_nodes)
        self.drop = nn.Dropout(0.3)

    def forward(self, x): # 按顺序摆放材料！
        # add sequence of convolutional and max pooling layers
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = F.relu(self.fc2(x))
        x = self.drop(x)
        x = self.fc3(x) 
        
        return x
model = Net(211,512,64,1)
print(model)




In [ ]:
# specify loss function
criterion = nn.MSELoss()

# specify optimizer
optimizer = optim.SGD(model.parameters(),lr = .02)

In [ ]:
n_epochs = 300 # you may increase this number to train a final model

valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for data, target in train_loader:
        # move tensors to GPU if CUDA is available
        
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        
        #print(target.size())
        #print(output.size())
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for data, target in valid_loader:
        # move tensors to GPU if CUDA is available
        
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {} \tValidation Loss: {}'.format(
        epoch, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({} --> {}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_wayfair.pt')
        valid_loss_min = valid_loss

In [ ]:
model.load_state_dict(torch.load('model_wayfair.pt'))

flatten = lambda l: [item for sublist in l for item in sublist]
true = []
pred = []
model.eval()
for data, target in valid_loader:
    # move tensors to GPU if CUDA is available
    #print(data.size())
    output = model(data)
    #print(output.view(1,-1).detach().numpy(),n)
    
    pred.append(output.detach().numpy())


In [ ]:
test = pd.read_csv('answer1.csv',index_col=0)
test.head()

In [ ]:
dummy_hold_regression = test[test['pred_convert_30'] == 1]

dummy_hold_regression.head()

In [ ]:
drop_list = ['pred_convert_30','numtasksotherone','numtasksotherthreeone','numtasksotherseventhree','numtasksotherthirtyseven',
             'numtasksothersixtythirty','numtasksotheryearsixty','decmakerflagone','decmakerflagsevenone',
             'decmakerflagfourteenseven','decmakerflagthirtyfourteen','currentstatus_Inactive','customersource_Display - Acquisition',
            'customersource_Display - Retargeting','customersource_Self ID']
data1= dummy_hold_regression.drop(drop_list,axis =1)

In [ ]:
X_tensor = torch.from_numpy(data1.values).float()
data_pre = [(x,0) for x in X_tensor]
batch_size = 32
num_workers = 0
testloader = torch.utils.data.DataLoader(data_pre, batch_size=batch_size, num_workers=num_workers)

In [ ]:
pred = []
model.eval()
for data, target in testloader:
    # move tensors to GPU if CUDA is available
    #print(data.size())
    output = model(data)
    #print(output.view(1,-1).detach().numpy(),n)

    pred.append(output.detach().numpy())
prediction = flatten(pred)
data1['pred_revenue_30'] = prediction
ans2 = data1[['pred_revenue_30']]

In [ ]:
answer = pd.read_csv('final_answer_need.csv',index_col=0)
fianl_answer = answer.join(ans2).replace(np.NaN,0)
fianl_answer.to_csv('submission.csv')